In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from IPython.display import display, Image
import PIL.Image as Image
from io import BytesIO

# Initialize MediaPipe Pose
pose_model = mp.solutions.pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Function to calculate Euclidean distance between two landmarks
def calculate_distance(landmark1, landmark2):
    return np.sqrt((landmark2.x - landmark1.x) ** 2 + (landmark2.y - landmark1.y) ** 2)

# Pose estimation and tracking function
def estimate_pose_and_track_movement(frame, previous_landmarks):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose_model.process(frame_rgb)
    movements = {}

    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)
        current_landmarks = results.pose_landmarks.landmark

        # Calculate movements for all landmarks
        if previous_landmarks:
            for i, landmark in enumerate(current_landmarks):
                landmark_name = mp.solutions.pose.PoseLandmark(i).name
                movements[landmark_name] = calculate_distance(landmark, previous_landmarks[i])

    return frame, current_landmarks, movements

# Function to display an image in Jupyter notebook
def display_frame(frame, frame_number, movements):
    print(f"Displaying frame {frame_number}")
    for landmark_name, movement in movements.items():
        print(f"{landmark_name} moved {movement:.2f}")
    _, encoded_image = cv2.imencode('.png', frame)
    i = Image.open(BytesIO(encoded_image.tobytes()))
    display(i)

# Overlay and export function with pose tracking at specific time intervals
def overlay_pose_and_export(video_path, output_path, time_interval=1.0):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    interval_frame_count = int(fps * time_interval)

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    frame_count = 0
    previous_landmarks = None
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        frame_with_pose, current_landmarks, movements = estimate_pose_and_track_movement(frame, previous_landmarks)
        
        # Update previous landmarks and display movements at specified intervals
        if frame_count % interval_frame_count == 0:
            previous_landmarks = current_landmarks
            display_frame(frame_with_pose, frame_count, movements)

        out.write(frame_with_pose)

    cap.release()
    out.release()
    pose_model.close()

    print(f"Processed video saved to {output_path}. Total frames processed: {frame_count}")

# Example usage
# overlay_pose_and_export('input_video.mp4', 'output_video.mp4', time_interval=1.0)
